In [ ]:
#드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt

In [ ]:
obs_cloud_filled = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/동서발전 태양광 발전량 예측 AI 경진대회/data/RF_cloud_2.csv')
energy = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/동서발전 태양광 발전량 예측 AI 경진대회/data/energy.csv')
site_info = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/동서발전 태양광 발전량 예측 AI 경진대회/data/site_info.csv')
obs = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/동서발전 태양광 발전량 예측 AI 경진대회/data/dangjin_obs_data.csv')
obs_cloud_filled2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/동서발전 태양광 발전량 예측 AI 경진대회/data/RF_cloud.csv')


In [ ]:
obs_cloud_filled

In [ ]:
obs_cloud_filled2

In [ ]:
obs_cloud_filled['Time'] = obs['일시']

In [ ]:
obs_cloud_filled=obs_cloud_filled.dropna(axis=0).reset_index(drop=True)

In [ ]:
obs_cloud_filled['energy'] = energy['dangjin']

In [ ]:
obs_cloud_filled.isnull().sum()

In [ ]:
obs_cloud_filled

In [ ]:
obs_cloud_filled['energy'] = energy['dangjin']


In [ ]:
obs_cloud_filled=obs_cloud_filled.drop(['Time'],axis=1)

In [ ]:
obs_cloud_filled

In [ ]:
obs_cloud_filled.columns

In [ ]:
from sklearn.model_selection import train_test_split
 
x = obs_cloud_filled[['Temperature', 'WindSpeed', 'Humidity', 'Rain', 'solar', 'sunTime',
       'Cloud', 'so2', 'ozone', 'no2', 'PM2.5', 'PM10']]
y = obs_cloud_filled[['energy']]
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3)

In [ ]:
#LightGBM Custom Metric
def nmae_10(y_pred, dataset):
    y_true = dataset.get_label()
    
    absolute_error = abs(y_true - y_pred)
    absolute_error /= capacity
    
    target_idx = np.where(y_true>=capacity*0.1)
    
    nmae = 100 * absolute_error[target_idx].mean()
    
    return 'score', nmae, False

In [ ]:
def sola_nmae(answer, pred):
    absolute_error = np.abs(answer - pred)
    
    absolute_error /= capacity
    
    target_idx = np.where(answer>=capacity*0.1)
    
    nmae = 100 * absolute_error[target_idx].mean()
    
    return nmae


In [ ]:
params = {
    'learning_rate': 0.01,
    'objective': 'regression',
    'metric':'mae',
    'seed':42
}


In [ ]:
from sklearn.model_selection import train_test_split
 
x = obs_cloud_filled[['Temperature', 'WindSpeed', 'Humidity', 'Rain', 'solar', 'sunTime',
       'Cloud', 'so2', 'ozone', 'no2', 'PM2.5', 'PM10']]
y = obs_cloud_filled[['energy']]
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3)

In [ ]:
train_x, train_y, val_x, val_y = obs_cloud_filled(y, x, target='energy')
train_dataset = lgb.Dataset(train_x, train_y)
val_dataset = lgb.Dataset(val_x, val_y)
capacity = 1000
dangjin_floating_model = lgb.train(params, train_dataset, 10000, val_dataset, feval=nmae_10, verbose_eval=500, early_stopping_rounds=100)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics    
from sklearn.metrics import classification_report 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

forest = RandomForestClassifier(n_estimators=100, random_state=10)
#,max_depth=16,  min_samples_leaf= 2, min_samples_split= 2

model = forest.fit(x_train, y_train)
y_pred = forest.predict(x_test)
print(y_pred)
print(y_test)

print('정확도 :', metrics.accuracy_score(y_test, y_pred)) 
print(mean_squared_error(y_test, y_pred))

In [ ]:
#obs_cloud_filled2


# LigthGBM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import lightgbm as lgb

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# Importing the dataset
x = obs_cloud_filled[['Temperature', 'WindSpeed', 'Humidity', 'Rain', 'solar', 'sunTime',
       'Cloud', 'so2', 'ozone', 'no2', 'PM2.5', 'PM10']]
y = obs_cloud_filled[['energy']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [ ]:
train_ds = lgb.Dataset(x_train, label = y_train) 
test_ds = lgb.Dataset(x_test, label = y_test) 

In [ ]:
#파라미터 튜닝
params = {'learning_rate': 0.01, 
          'max_depth': 16, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'mse', 
          'is_training_metric': True, 
          'num_leaves': 144, 
          'feature_fraction': 0.9, 
          'bagging_fraction': 0.7, 
          'bagging_freq': 5, 
          'seed':2018}

In [ ]:
model = lgb.train(params, train_ds, 1000, test_ds, verbose_eval=100, early_stopping_rounds=100)

In [ ]:
predict_train = model.predict(x_train)
predict_test = model.predict(x_test)

In [ ]:
mse = mean_squared_error(y_test, predict_test)
r2 = r2_score(y_test, predict_test)

In [ ]:
print('Mean squared error: ', mse)
print('R2 score: ', r2)  #R2 score:  0.3673130671989723
